In [1]:
# analyze the results of the local sensitivity runs

In [2]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# load the csv to get the ignition delay times
# csv_file = '/scratch/harris.se/autoscience/local_sensitivity/practice.csv'
csv_file = '/scratch/harris.se/autoscience/local_sensitivity/ignition_delays_007.csv'
df = pd.read_csv(csv_file, skiprows=1, names=['model', 'delay'])

In [20]:
base_delay

0.14857534592261107

In [21]:
df['delay'].values[324]

0.14857534592261107

In [4]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'

cti_path = os.path.join(basedir, 'chem_annotated.cti')
gas = ct.Solution(cti_path)

In [5]:
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)


In [6]:
# get the mapping between RMG and Cantera reactions
# Species Diff
common_species = []
RMG2CT_mapping = {}
CT2RMG_mapping = {}
for i, rmg_sp in enumerate(species_list):
    for j, ct_sp in enumerate(gas.species()):
        if str(rmg_sp) == ct_sp.name:
            common_species.append([rmg_sp, ct_sp])
            RMG2CT_mapping[i] = j
            CT2RMG_mapping[j] = i
            break


def ct_matches_rmg_rxn(rmg_rxn, ct_rxn):
    rmg_reactants = set()
    ct_reactants = set()
    for key in ct_rxn.reactants.keys():
        ct_reactants.add(key)
    for reactant in rmg_rxn.reactants:
        rmg_reactants.add(str(reactant))
    if rmg_reactants != ct_reactants:
        return False
    
    rmg_products = set()
    ct_products = set()
    for key in ct_rxn.products.keys():
        ct_products.add(key)
    for product in rmg_rxn.products:
        rmg_products.add(str(product))
    if rmg_products != ct_products:
        return False

    # check duplicates
    if not ct_rxn.duplicate:
        return True
    
    if type(rmg_rxn.kinetics) == rmgpy.kinetics.arrhenius.Arrhenius:
        return np.isclose(rmg_rxn.to_cantera().rate.pre_exponential_factor, ct_rxn.rate.pre_exponential_factor)
    
#     if type(rmg_rxn.kinetics) == rmgpy.kinetics.arrhenius.MultiArrhenius
    
    return False
#     if type(ct_rxn) == list:
#         return False
#     if type(rmg_rxn.to_cantera()) == list:
#         return False
    
#     if type(rmg_rxn.to_cantera() == ct._cantera.FalloffReaction):
#         return False
#     # check the rate
#     return rmg_rxn.to_cantera().rate.pre_exponential_factor == ct_rxn.rate.pre_exponential_factor
    
    
            
# Reaction Diff
common_reactions = []
RMG2CT_rxn_mapping = {}
CT2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(reaction_list):
    for j, ct_rxn in enumerate(gas.reactions()):
        if ct_matches_rmg_rxn(rmg_rxn, ct_rxn):
            RMG2CT_rxn_mapping[i] = j
            CT2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, ct_rxn])
            break
    else:
        print(f'No match for {reaction_list.index(rmg_rxn)} {rmg_rxn}, {type(rmg_rxn.kinetics)}')
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

No match for 1 O(5) + H2(13) <=> H(14) + OH(15), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 18 HO2(16) + HO2(16) <=> O2(2) + H2O2(17), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 23 OH(15) + H2O2(17) <=> HO2(16) + H2O(8), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 53 O2(2) + C2H3(22) <=> OH(15) + CH2CO(24), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 60 CH(3) + CH2CHO(21) <=> HCO(19) + C2H3(22), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 83 CH2CHO(21) <=> CO(6) + CH3(18), <class 'rmgpy.kinetics.falloff.Troe'>
No match for 98 OH(15) + CO(6) <=> H(14) + CO2(7), <class 'rmgpy.kinetics.arrhenius.MultiArrhenius'>
No match for 110 O2(2) + C2H3(22) <=> CO2(7) + CH3(18), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 139 O2(2) + C2H3(22) <=> H(14) + CO(6) + CH2O(9), <class 'rmgpy.kinetics.arrhenius.MultiPDepArrhenius'>
No match for 148 O(5) + CH2CHO(21) <=> HCO(19) + CH2O(9

In [ ]:
reaction_list[1].to_cantera()

In [ ]:
gas.reactions()[3]

In [ ]:
len(reaction_list)

In [ ]:
len(gas.reactions())

In [ ]:
RMG2CT_rxn_mapping[1].

In [ ]:
CT2RMG_rxn_mapping[1]

In [7]:
# find the estimated reactions
libraries = [
    'BurkeH2O2inN2',
    'CurranPentane',
    'FFCM1(-)',
    'combustion_core/version5',
    'Klippenstein_Glarborg2016',
    'C2H4+O_Klipp2017',
]

rmg_estimated = []
for i in range(0, len(reaction_list)):
    if not hasattr(reaction_list[i], 'family'):
        continue
    if reaction_list[i].family in libraries:
        continue
    rmg_estimated.append(i)



In [25]:
reaction_list[288].kinetics

Arrhenius(A=(1.054e+10,'cm^3/(mol*s)'), n=0.97, Ea=(1.586,'kcal/mol'), T0=(1,'K'))

In [10]:
base_delay = 0.14857534592261107

In [11]:
# Load the covariance matrix
covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)

In [12]:
delays = df['delay'].values
for i in range(0, len(df)):
    if delays[i] == -1:
        delays[i] = base_delay
    

In [ ]:
np.min(df['delay'])

In [ ]:
np.max(df['delay'].values)

In [13]:
uncertainties_ct = np.zeros(len(gas.reactions()))
for i in range(0, len(uncertainties_ct)):
    try:
        rmg_index = CT2RMG_rxn_mapping[i]
        uncertainties_ct[i] = Sigma_k[rmg_index, rmg_index]
    except KeyError:
        continue

In [ ]:
len(reaction_list)

0.0

In [14]:
# Get the top 50:




differences = np.abs(delays - base_delay)
# multiply sensitivity times uncertainty

score = np.multiply(differences, uncertainties_ct)

best_runs = [x for _, x in sorted(zip(score, np.arange(0, len(gas.reactions()))))][::-1]

In [15]:
# print the top reactions
print('idx', '\t', 'ln k', '\t', 'prod', '\t', 'family', '\t', 'reaction')


for i in range(0, len(gas.reactions())):
    try:
        rmg_index = CT2RMG_rxn_mapping[best_runs[i]]
        if rmg_index not in rmg_estimated:
            continue
    except KeyError:
        print(f'Cantera reaction mapping not found for {best_runs[i]} {gas.reactions()[best_runs[i]]}')
        continue
    delta = np.round(Sigma_k[rmg_index][rmg_index], 1)
    
#     print(rmg_index, '\t', delta, '\t', np.round(score[best_runs[i]],4), '\t', reaction_list[rmg_index].family, '\t', reaction_list[rmg_index])
    print(rmg_index, '\t', delta, '\t', np.round(score[best_runs[i]],4), '\t', reaction_list[rmg_index].family, '\t', reaction_list[rmg_index])

idx 	 ln k 	 prod 	 family 	 reaction
324 	 3.6 	 0.003 	 intra_H_migration 	 S(229) <=> S(225)
915 	 10.7 	 0.0025 	 Disproportionation 	 O2(2) + S(777) <=> HO2(16) + S(252)
748 	 3.2 	 0.0019 	 H_Abstraction 	 HCO(19) + C4H8(189) <=> CH2O(9) + C4H7(190)
1287 	 3.9 	 0.0015 	 H_Abstraction 	 C3H6O3(73) + SC4H9(183) <=> C3H5O3(72) + butane(1)
749 	 11.0 	 0.0013 	 H_Abstraction 	 HCO(19) + C4H8(188) <=> CH2O(9) + C4H7(190)
809 	 1.8 	 0.001 	 H_Abstraction 	 PC4H9(182) + S(787) <=> S(225) + butane(1)
419 	 10.7 	 0.0008 	 Disproportionation 	 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24)
213 	 3.2 	 0.0005 	 Disproportionation 	 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
1288 	 4.3 	 0.0005 	 H_Abstraction 	 C3H6O3(73) + PC4H9(182) <=> C3H5O3(72) + butane(1)
805 	 1.8 	 0.0004 	 H_Abstraction 	 PC4H9(182) + S(787) <=> S(229) + butane(1)
370 	 4.3 	 0.0004 	 H_Abstraction 	 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
793 	 1.8 	 0.0003 	 Disproportionation 	 HO2(16) + C4H7(190) <

1182 	 1.0 	 0.0 	 H_Abstraction 	 C3H5-A(94) + C4H8(188) <=> C3H6(12) + C4H7(192)
1680 	 10.7 	 0.0 	 Disproportionation 	 C2H4O(703) + C2H5O2(47) <=> CH2CHO(21) + C2H6O2(48)
828 	 1.8 	 0.0 	 H_Abstraction 	 C2H3(22) + S(787) <=> C2H4(11) + S(229)
912 	 1.0 	 0.0 	 Birad_R_Recombination 	 O(5) + S(777) <=> S(229)
1797 	 3.6 	 0.0 	 H_Abstraction 	 H2O2(17) + C4H8(748) <=> HO2(16) + SC4H9(183)
217 	 9.2 	 0.0 	 Disproportionation 	 C2H3(22) + CH2CHO(21) <=> CH2CO(24) + C2H4(11)
1218 	 10.7 	 0.0 	 Disproportionation 	 OH(15) + C3H5O(129) <=> H2O(8) + C3H4O(74)
1547 	 10.7 	 0.0 	 Disproportionation 	 HO2(16) + OCHO(38) <=> H2O2(17) + CO2(7)
595 	 10.7 	 0.0 	 Disproportionation 	 O(5) + CH2CHO(21) <=> OH(15) + CH2CO(24)
1117 	 10.7 	 0.0 	 Disproportionation 	 CH3O2(45) + S(777) <=> CH3O2H(46) + S(252)
1756 	 3.6 	 0.0 	 Disproportionation 	 CH3CO(20) + S(186) <=> CH2CO(24) + S(187)
1102 	 1.8 	 0.0 	 H_Abstraction 	 CH3O2H(46) + S(238) <=> CH3O2(45) + S(787)
769 	 1.0 	 0.0 	 H_Abstr

438 	 1.8 	 0.0 	 Disproportionation 	 CH2CHO(21) + IC3H7(93) <=> CH3CHO(35) + C3H6(12)
1567 	 10.7 	 0.0 	 Disproportionation 	 OCHO(38) + CH3O2(45) <=> CO2(7) + CH3O2H(46)
738 	 2.8 	 0.0 	 Disproportionation 	 C2H4O(703) + C4H7(190) <=> CH2CHO(21) + C4H8(188)
1398 	 1.8 	 0.0 	 Disproportionation 	 C4H6(2534) + C4H7(190) <=> C4H6(194) + C4H7(190)
1471 	 1.0 	 0.0 	 H_Abstraction 	 C2H6(32) + S(225) <=> C2H5(33) + S(787)
1601 	 1.8 	 0.0 	 Disproportionation 	 OCHO(38) + C4H7(190) <=> HOCHO(40) + C4H6(194)
1592 	 1.8 	 0.0 	 Disproportionation 	 OCHO(38) + SC4H9(183) <=> HOCHO(40) + C4H8(188)
1514 	 2.8 	 0.0 	 H_Abstraction 	 CH3O(31) + CH3O2H(46) <=> CH3O2(45) + CH3OH(30)
991 	 1.8 	 0.0 	 Disproportionation 	 C4H7(190) + C4H8(748) <=> C4H6(194) + SC4H9(183)
752 	 6.3 	 0.0 	 Disproportionation 	 CH2(23) + C4H8(748) <=> CH3(18) + C4H7(190)
929 	 3.9 	 0.0 	 Disproportionation 	 C2H5(33) + S(777) <=> C2H6(32) + S(252)
1510 	 2.8 	 0.0 	 H_Abstraction 	 CH3O(31) + S(787) <=> CH3OH(30

In [ ]:
reaction_index = 915
print('Reactants:')
for reactant in reaction_list[reaction_index].reactants:
    print('\t', reactant.smiles)

print('Products:')
for product in reaction_list[reaction_index].products:
    print('\t', product.smiles)

print(Sigma_k[reaction_index][reaction_index])